In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [8]:
#Merge ther 12 months of Sales data into Single CSV file

files = [file for file in os.listdir('..//SALES_2019//data')]
all_data = pd.DataFrame()
for file in files:
    data = pd.read_csv('..//SALES_2019//data/' + file)
    all_data = pd.concat([all_data, data])
all_data.to_csv('all_data', index = False)


In [9]:
#Data preprosessing
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 186850 entries, 0 to 11685
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          186305 non-null  object
 1   Product           186305 non-null  object
 2   Quantity Ordered  186305 non-null  object
 3   Price Each        186305 non-null  object
 4   Order Date        186305 non-null  object
 5   Purchase Address  186305 non-null  object
dtypes: object(6)
memory usage: 10.0+ MB


In [10]:
categorical = all_data.select_dtypes(['category','object']).columns

for col in categorical:
    print('{} : {} unique value(s)'.format(col, all_data[col].nunique()))


Order ID : 178438 unique value(s)
Product : 20 unique value(s)
Quantity Ordered : 10 unique value(s)
Price Each : 18 unique value(s)
Order Date : 140323 unique value(s)
Purchase Address : 140788 unique value(s)


In [11]:
missing_val = all_data.isnull().sum()

In [12]:
total_cell = np.product(all_data.shape)
total_missing = missing_val.sum()
percentage_missing = (total_missing/total_cell)*100
print(f"{percentage_missing:.2f}%")

0.29%


In [13]:
#Drop NaN data
all_data = all_data.dropna(how='all')
all_data[all_data.isna().any(axis=1)]
all_data.head()

,Order ID,Product,Quantity Ordered,Price Each,Order Date,Purchase Address
0,176558,USB-C Charging Cable,2,11.95,19-04-19,"917 1st St, Dallas, TX 75001"
2,176559,Bose SoundSport Headphones,1,99.99,07-04-19,"682 Chestnut St, Boston, MA 02215"
3,176560,Google Phone,1,600,12-04-19,"669 Spruce St, Los Angeles, CA 90001"
4,176560,Wired Headphones,1,11.99,12-04-19,"669 Spruce St, Los Angeles, CA 90001"
5,176561,Wired Headphones,1,11.99,30-04-19,"333 8th St, Los Angeles, CA 90001"


In [14]:
all_data = all_data[all_data['Quantity Ordered'].astype('string').str.contains("Quantity Ordered")==False]
all_data['Quantity Ordered'] = all_data['Quantity Ordered'].astype('int64')
all_data['Price Each'] = all_data['Price Each'].astype('float')


In [15]:
all_data['Order Date'] = pd.to_datetime(all_data['Order Date'])
all_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 185950 entries, 0 to 11685
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   Order ID          185950 non-null  object        
 1   Product           185950 non-null  object        
 2   Quantity Ordered  185950 non-null  int64         
 3   Price Each        185950 non-null  float64       
 4   Order Date        185950 non-null  datetime64[ns]
 5   Purchase Address  185950 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(3)
memory usage: 9.9+ MB


In [16]:
categorical = all_data.select_dtypes(['category','object']).columns

for col in categorical:
    print('{} : {} unique value(s)'.format(col, all_data[col].nunique()))

Order ID : 178437 unique value(s)
Product : 19 unique value(s)
Purchase Address : 140787 unique value(s)
